In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

events = pd.read_csv('events_up_to_01062018.csv')
training = pd.read_csv('labels_training_set.csv')

/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [5]:
eventsWithLabel = events.merge(training, how='left', left_on='person', right_on='person')
eventsWithLabel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
eventsWithLabel['timestamp'] = pd.to_datetime(eventsWithLabel['timestamp'])

In [7]:
# groupbyPersonas = eventsWithLabel.loc[:,['person', 'event']].groupby('person')
# dicc = {}
# for clave, data in groupbyPersonas:
#     lista = []
#     lista = data.drop_duplicates(subset = 'event', keep = 'first')['event'].tolist()
#     dicc[clave] = lista

In [8]:
#Obtengo una lista de todos los eventos que tiene esa persona
# eventsWithLabel['eventos'] = eventsWithLabel['person'].map(lambda x: dicc[x])
#Obtengo los dias de la semana
eventsWithLabel['dayweek'] = eventsWithLabel.timestamp.dt.dayofweek
#Obtengo la fecha
eventsWithLabel['fecha'] = eventsWithLabel.timestamp.dt.date
#Separo la marca
eventsWithLabel['marca'] = eventsWithLabel['model'].map(lambda x: str(x).split()[0])

In [9]:
#cantidad de personas que ingresaron 27624.
groupbyCantEventos = eventsWithLabel.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos.sort_values(by = 'cantEventos', ascending = False)
eventsWithLabel = pd.merge(eventsWithLabel, groupbyCantEventos, on = 'person')

In [10]:
#Filtro a la gente que no tiene el evento visited_site
personasConVisitas = eventsWithLabel.loc[eventsWithLabel['event'] == 'visited site']\
                             .drop_duplicates(subset = 'person', keep = 'first')['person'].tolist()
dataNotVisit = eventsWithLabel.loc[eventsWithLabel['person'].isin(personasConVisitas)]

In [11]:
eventsWithLabel2 = dataNotVisit.loc[dataNotVisit['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

In [12]:
eventsWithLabel3 = pd.merge(dataNotVisit, eventsWithLabel2, on = ['person', 'fecha'])
eventsWithLabel3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
eventsWithLabel3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

In [13]:
def obtenerSistema(so):
    sistema = so.split()
    if len(sistema) > 1:
        otro = sistema[1].split('.')[0]
        return sistema[0] + ' ' + otro
    else:
        return sistema[0]

In [14]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].map(obtenerSistema)

In [15]:
eventosUnicos = eventsWithLabel3.drop_duplicates(subset="event", keep="first").reset_index()["event"]

In [16]:
# label_encoder = LabelEncoder()
# label_encoder.fit(eventosUnicos)
# integer_encoded = label_encoder.transform(eventsWithLabel3["event"])
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


In [17]:
#Funcion que realiza el oneHotEncoding
def oneHotEncoding(column, uniqueArray, dataFrame):
    label_encoder = LabelEncoder()
    label_encoder.fit(uniqueArray)
    integer_encoded = label_encoder.transform(dataFrame[column])
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    for i, item in enumerate(uniqueArray):
        dataFrame[item] = onehot_encoded[:, label_encoder.transform(uniqueArray)[i]]


In [18]:
oneHotEncoding("event", eventosUnicos, eventsWithLabel3)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
# for i, event in enumerate(eventosUnicos):
#     eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(eventosUnicos)[i]]

In [20]:
eventsWithLabel3.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,viewed product,search engine hit,checkout,searched products,generic listing,visited site,ad campaign hit,brand listing,lead,conversion
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
eventsWithLabel3["color"] = eventsWithLabel3.color.str.split(" ", expand=True)

In [22]:
top20Colors = eventsWithLabel3['color'].value_counts().nlargest(20).index
top20Colors

Index(['Preto', 'Dourado', 'Cinza', 'Branco', 'Prateado', 'Ouro', 'Rosa',
       'Prata', 'Azul', 'Platinum', 'Ametista', 'Verde', 'Vermelho', 'Bambu',
       'Black', 'Titânio', 'Indigo', 'Amarelo', 'Cabernet', 'Olympic'],
      dtype='object')

In [23]:
def colorGrouping(x):
    return x if x in top20Colors else 'Otro'

In [24]:
eventsWithLabel3['color'] = eventsWithLabel3['color'].apply(colorGrouping)

In [25]:
eventsWithLabel3['color'] = eventsWithLabel3['color'].fillna(value='idk')
coloresUnicos = eventsWithLabel3.drop_duplicates(subset='color', keep='first').reset_index()["color"]

In [26]:
oneHotEncoding("color", coloresUnicos, eventsWithLabel3)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [27]:
eventsWithLabel3['condition'].value_counts()

Bom                   541428
Excelente             374870
Muito Bom             354901
Bom - Sem Touch ID     31391
Novo                    3397
Name: condition, dtype: int64

In [28]:
def conditionType(x):
    conditions = {
        "Bom - Sem Touch ID" : 1,
        "Bom" : 2,
        "Muito Bom" : 3,
        "Excelente" : 4,
        "Novo" : 5
    }
    return conditions[x] if x in conditions else 0

In [29]:
eventsWithLabel3['condition'] = eventsWithLabel3['condition'].apply(conditionType)

In [30]:
eventsWithLabel3['storage'].value_counts()

16GB     436689
32GB     425751
64GB     226577
128GB     98510
8GB       94320
256GB     17501
4GB        5318
512MB      1321
Name: storage, dtype: int64

In [31]:
def storageType(x):
    storages = {
        "512MB" : 1,
        "4GB" : 2,
        "8GB" : 3,
        "16GB" : 4,
        "32GB" : 5,
        "64GB" : 6,
        "128GB" : 7,
        "256GB" : 8
    }
    return storages[x] if x in storages else 0

In [32]:
eventsWithLabel3['storage'] = eventsWithLabel3['storage'].apply(storageType)

In [33]:
eventsWithLabel3['model'].value_counts()

iPhone 6                                     106118
iPhone 5s                                    100448
iPhone 6S                                     96485
iPhone 7                                      58598
Samsung Galaxy S7 Edge                        49915
iPhone 7 Plus                                 44627
Samsung Galaxy S7                             39279
iPhone 6S Plus                                37653
iPhone SE                                     36671
Samsung Galaxy S8                             32513
iPhone 6 Plus                                 32000
Samsung Galaxy S6 Edge                        30555
Samsung Galaxy J5                             29121
iPhone 5c                                     27763
Samsung Galaxy S6 Flat                        27485
Motorola Moto G4 Plus                         24888
Samsung Galaxy J7 Prime                       24808
Samsung Galaxy S8 Plus                        20804
iPhone 5                                      18388
Samsung Gala

In [34]:
eventsWithLabel3['model'] = eventsWithLabel3['model'].fillna(value='idk')
modelosUnicos = eventsWithLabel3.drop_duplicates(subset='model', keep='first').reset_index()['model']


In [36]:
def most_viewed_model(group):
    top_2_most_viewed = group.value_counts().head(2)
    if top_2_most_viewed.index[0] != "idk" or len(top_2_most_viewed) == 1:
        return top_2_most_viewed.index[0]
    else:
        return top_2_most_viewed.index[1]
    
e = eventsWithLabel3.groupby('person').model.apply(most_viewed_model).reset_index()
e = e.rename(columns=dict(level_2='model', model='most_viewed_model'))
e
eventsWithLabel3 = eventsWithLabel3.merge(e, how='inner', left_on='person', right_on='person')
eventsWithLabel3.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,Titânio,Prata,Azul,Verde,Indigo,Ametista,Black,Cabernet,Amarelo,most_viewed_model
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,5,Dourado,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Samsung Galaxy J7 Prime
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,5,Dourado,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Samsung Galaxy J7 Prime
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,idk,0,0,Otro,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Samsung Galaxy J7 Prime
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,5,Dourado,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Samsung Galaxy J7 Prime
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,3,5,Dourado,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Samsung Galaxy J7 Prime


In [ ]:
label_encoder.fit(modelosUnicos['model'])
eventsWithLabel3['model'] = label_encoder.fit_transform(eventsWithLabel3['model'])

In [ ]:
eventsWithLabel3['search_term'] = eventsWithLabel3['search_term'].fillna(value='idk')
searchTermUnicos = eventsWithLabel3.drop_duplicates(subset='search_term', keep='first').reset_index()
searchTermUnicos['search_term']

In [ ]:
label_encoder.fit(searchTermUnicos['search_term'])
eventsWithLabel3['search_term'] = label_encoder.fit_transform(eventsWithLabel3['search_term'])

In [ ]:
eventsWithLabel3['campaign_source'].value_counts()

In [ ]:
eventsWithLabel3['campaign_source'] = eventsWithLabel3['campaign_source'].fillna(value='idk')
campaingUnicos = eventsWithLabel3.drop_duplicates(subset='campaign_source', keep='first').reset_index()
campaingUnicos['campaign_source']

In [ ]:
label_encoder.fit(campaingUnicos['campaign_source'])
eventsWithLabel3['campaign_source'] = label_encoder.fit_transform(eventsWithLabel3['campaign_source'])

In [ ]:
eventsWithLabel3['search_engine'].value_counts()

In [ ]:
eventsWithLabel3['search_engine'] = eventsWithLabel3['search_engine'].fillna(value='idk')
searchEngineUnicos = eventsWithLabel3.drop_duplicates(subset='search_engine', keep='first').reset_index()["search_engine"]

In [ ]:
label_encoder.fit(searchEngineUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["search_engine"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(searchEngineUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(searchEngineUnicos)[i]]

In [ ]:
eventsWithLabel3['browser_version'] = eventsWithLabel3['browser_version'].fillna(value='idk')
browserUnicos = eventsWithLabel3.drop_duplicates(subset='browser_version', keep='first').reset_index()
browserUnicos['browser_version']

In [ ]:
label_encoder.fit(browserUnicos['browser_version'])
eventsWithLabel3['browser_version'] = label_encoder.fit_transform(eventsWithLabel3['browser_version'])

In [ ]:
marcaUnicos = eventsWithLabel3.drop_duplicates(subset='marca', keep='first').reset_index()["marca"]

In [ ]:
label_encoder.fit(marcaUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["marca"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(marcaUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(marcaUnicos)[i]]

In [ ]:
eventsWithLabel3['city'].value_counts()

In [ ]:
eventsWithLabel3['city'] = eventsWithLabel3['city'].fillna(value='idk')
ciudadUnicos = eventsWithLabel3.drop_duplicates(subset='city', keep='first').reset_index()
ciudadUnicos['city']

In [ ]:
label_encoder.fit(ciudadUnicos['city'])
eventsWithLabel3['city'] = label_encoder.fit_transform(eventsWithLabel3['city'])

In [ ]:
eventsWithLabel3['region'].value_counts()

In [ ]:
eventsWithLabel3['region'] = eventsWithLabel3['region'].fillna(value='idk')
regionUnicos = eventsWithLabel3.drop_duplicates(subset='region', keep='first').reset_index()
regionUnicos['region']

In [ ]:
label_encoder.fit(regionUnicos['region'])
eventsWithLabel3['region'] = label_encoder.fit_transform(eventsWithLabel3['region'])

In [ ]:
eventsWithLabel3['country'].value_counts()

In [ ]:
eventsWithLabel3['country'] = eventsWithLabel3['country'].fillna(value='idk')
paisUnicos = eventsWithLabel3.drop_duplicates(subset='country', keep='first').reset_index()
paisUnicos['country']

In [ ]:
label_encoder.fit(paisUnicos['country'])
eventsWithLabel3['country'] = label_encoder.fit_transform(eventsWithLabel3['country'])

In [ ]:
deviceUnicos = eventsWithLabel3.drop_duplicates(subset='device_type', keep='first').reset_index()["device_type"]

In [ ]:
label_encoder.fit(deviceUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["device_type"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(deviceUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(deviceUnicos)[i]]

In [ ]:
eventsWithLabel3['SO'].value_counts()

In [ ]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].fillna(value='idk')
SOUnicos = eventsWithLabel3.drop_duplicates(subset='SO', keep='first').reset_index()
SOUnicos['SO']

In [ ]:
label_encoder.fit(SOUnicos['SO'])
eventsWithLabel3['SO'] = label_encoder.fit_transform(eventsWithLabel3['SO'])

In [ ]:
eventsWithLabel3['channel'].value_counts()

In [ ]:
channelUnicos = eventsWithLabel3.drop_duplicates(subset='channel', keep='first').reset_index()["channel"]

In [ ]:
label_encoder.fit(channelUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["channel"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(channelUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(channelUnicos)[i]]

In [ ]:
eventsWithLabel3['person'] = eventsWithLabel3['person'].fillna(value='idk')
personUnicos = eventsWithLabel3.drop_duplicates(subset='person', keep='first').reset_index()
personUnicos['person']

In [ ]:
label_encoder.fit(personUnicos['person'])
eventsWithLabel3['person'] = label_encoder.fit_transform(eventsWithLabel3['person'])

In [ ]:
eventsWithLabel3['fecha'] = eventsWithLabel3['fecha'].fillna(value='idk')
fechaUnicos = eventsWithLabel3.drop_duplicates(subset='fecha', keep='first').reset_index()
fechaUnicos['fecha']

In [ ]:
label_encoder.fit(fechaUnicos['fecha'])
eventsWithLabel3['fecha'] = label_encoder.fit_transform(eventsWithLabel3['fecha'])

In [ ]:
eventsWithLabel3.info()

In [ ]:
eventsWithLabel3.head()

In [ ]:
testDF = eventsWithLabel3.loc[eventsWithLabel3['label'].isnull()]
testDF.head()

In [ ]:
trainingDF = eventsWithLabel3.loc[eventsWithLabel3['label'].notnull()]
trainingDF.head()

In [ ]:
trainingDF['storage'].value_counts()

In [ ]:
trainingDF['color'].value_counts()

In [ ]:
trainingDF['campaign_source'].value_counts()

In [ ]:
trainingDF['search_engine'].value_counts()

In [ ]:
trainingDF.info()

In [ ]:
label_array = np.array(trainingDF['label'])
data_array = np.array(trainingDF.drop(['label', 'static_page'], axis=1))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_array, label_array, test_size = 0.30, random_state = 1)

In [ ]:
def logisticReg(x_train, x_test, y_train, y_test):
    logReg = LogisticRegression(solver='sag', random_state=1)
    logReg.fit(x_train, y_train)
    y_pred = logReg.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def knn(k, x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def decisionTree(max_depth, max_features, x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    dt.fit(x_train, y_train)
    y_pred = dt.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def xgboost(max_depth, n_estimators, x_train, x_test, y_train, y_test):
    xgb = XGBClassifier(max_depth=max_depth, n_estimators=n_estimators)
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
result = logisticReg(x_train, x_test, y_train, y_test)
result